In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import base
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import RidgeCV, LinearRegression, SGDRegressor, Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Recommendation Engine
<!-- requirement: projects/ml-subset -->

*N.B. This notebook takes both a lot of memory and a fair amount of time to run.  You should close other kernels before running this notebook, and you probably want to do a run all at the beginning.*

## Problem definition and data format

The goal of a recommendation engine is to match items to users that will prefer them.  Since we have a set of previous ratings, we can use them to make guesses about which items a user will rate highest.  In this tutorial, we will cover two ways of doing this:

1. **Content-based rating:** We can make this guess based only on features of the users or items.
1. **Collaborative Rating:** We could also look at ratings of similar users or similar items.

For an example data set, we are using the [MovieLens 10M dataset](http://files.grouplens.org/datasets/movielens/ml-10m.zip).  This contains about 10M ratings for 10k movies by 72k users.  We will be building applications that attempts to present movies that a given user would rate highly.  The following cell will download and extract the data to the `projects/ml-10M100K/` directory.

In [ ]:
%%bash
# Uncomment the following line to trigger the download of the data set.
#DO_DOWNLOAD=true
cd projects
if [ ! -d "ml-10M100K" ] && [ "$DO_DOWNLOAD" ] ; then
    wget http://files.grouplens.org/datasets/movielens/ml-10m.zip
    unzip ml-10m.zip
    rm ml-10m.zip
fi

In [ ]:
datadir = 'projects/ml-10M100K' if os.path.isdir('projects/ml-10M100K') else 'projects/ml-subset/'

There are three data files that are unpacked.  `movies.dat` contains movie IDs, titles, and categories.  `tags.dat` contains a list of tag applications; each line representing the application of a single tag to a movie by a user.  Finally, `ratings.dat` contains the ratings of the movies by the users.

Each of these files are in a custom format, with fields separated by `::`.  They're easy enough to parse with Python string processing.

In [ ]:
def parse_movie_line(l):
    id_, title, cats = l.strip().split('::')
    return {'id': int(id_), 'title': title, 'year': int(title.rsplit(' ')[-1][1:-1]), 
            'categories': cats.split('|')}

with open(os.path.join(datadir, 'movies.dat'), 'r') as f:
    df = pd.DataFrame([parse_movie_line(l) for l in f]).set_index('id')

df.head()

In [ ]:
def parse_tag_line(l):
    uid, mid, tag, time = l.strip().split('::')
    return {'user_id': int(uid), 'movie_id': int(mid), 'tag': tag}

with open(os.path.join(datadir, 'tags.dat'), 'r') as f:
    df_tags = pd.DataFrame([parse_tag_line(l) for l in f])

df_tags.head()

In [ ]:
def parse_rating_line(l):
    uid, mid, rating, time = l.strip().split('::')
    return {'user_id': int(uid), 'movie_id': int(mid), 'rating': float(rating)}

with open(os.path.join(datadir, 'ratings.dat'), 'r') as f:
    df_ratings = pd.DataFrame([parse_rating_line(l) for l in f])

df_ratings.head()

## Feature engineering

We will start with content modeling, based on the movie categories.  If a user rated a bunch of Adventure films highly, a simple scheme would be to recommend more Adventure films.  Of coure, many users may like several genres of movies, so we need a way to represent categories beyond just the string representing the favorite.

The first approach for vectorizing the categories might be to assign numbers to each of the categories.  For example, we could say *Adventure* = 1, *Drama* = 2, *Comedy* = 3, and so on.  This approach, however, signals that there is some order to these genres.  This mapping would suggest that *Drama* = (*Adventure* + *Comedy*)/2, which isn't meaningful.  We don't want a movie that is an *Adventure Comedy* to have the same feature as a *Drama*.

Instead, we need to recognize that genre is a categorical variable, and thus should be encoded with one-hot encoding.  Essentially, this give each genre its own dimension in feature space.  Our *Adventure Comedy* movie will be located along both the *Adventure* and *Comedy* axes, but at 0 along the *Drama* axis.

We will use *Scikit Learn's* transformers to do this conversion.  We don't know the categories *a priori*, as we did with the days of the week, so we will use the `DictVectorizer` to help us.  This transformer takes in a list of dictionaries.  Each key in the dictionaries gets mapped to a column, and the values for those keys are placed in the appropriate column.  Columns for keys that are not present in a particular row are filled with zeros.

In order to use this, we need to transform our list of genres in to a dictionary, with values of one.

In [ ]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        def to_dict(l):
            try:
                return {x: 1 for x in l}
            except TypeError:
                return {}
        
        return X[self.col].apply(to_dict)

A pipeline helps us chain transformers together.  Since the pipeline doesn't end in an estimator, it acts as a transformer instead.

In [ ]:
cat_pipe = Pipeline([('encoder', DictEncoder('categories')),
                     ('vectorizer', DictVectorizer())])
features = cat_pipe.fit_transform(df)
features

The `DictVectorizer` returns a sparse matrix.  This allows efficient operations even on high-dimensional data.

## Nearest Neighbors

Now that we have a way to describe a movie, we want to be able to find other movies that are nearby in feature space.  This is the **nearest neighbors** problem, and Scikit Learn provides a class to handle this.

When the `NearestNeighbors` object is fit, it records all of the rows in such a way that it can efficiently look through them to find which is nearest to one queried later.  The `NearestNeighbors` assumes that the closest points are the ones for which the [Minkowski Distance](https://en.wikipedia.org/wiki/Minkowski_distance) is minimized.  That is, if we have two rows of our feature vector $X_{i\cdot}$ and $X_{j\cdot}$, it minimizes
$$ \|X_{i\cdot} - X_{j\cdot}\|_p^p = \sum_k \left(X_{ik} - X_{jk}\right)^p $$
By default, it computes $p=2$, which is the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance).

In [ ]:
nn = NearestNeighbors(n_neighbors=20).fit(features)

We will discuss how to measure performance of these models later, but for now we will just eyeball the results.  We'll pick out two movies as test cases.  *Toy Story* happens to be the first movie in the data set.  *Dr. Strangelove* is a favorite of the author.

In [ ]:
df.iloc[[0, 737]]

The results for *Toy Story* are reasonable.  All of the movies are animated children's movies, and *Toy Story 2* shows up.  But by relying only on the genres, we are unable to select Pixar movies specifically, for example.

In [ ]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

This limitation is more apparent when looking a movies similar to *Dr. Strangelove*.  We get all the *War Comedy* movies, but that doesn't mean that they're similar to *Dr. Strangelove*.  While the author has never seen *The Wackiest Ship in the Army*, he is pretty sure it is of a rather different tone.

In [ ]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

## Tag Data

The problem is that we're only getting 20 bits of data about each movie from the categories.  We have much more information in the user-applied tags, though, so let's bring that to bear.

As a reminder, the tag data is stored one tag application per row.

In [ ]:
df_tags.head()

We want all tags for each movie, so we group by the movie_id and get a list of tags.

In [ ]:
all_tags = df_tags.groupby('movie_id')['tag'].apply(lambda x: x.tolist())
all_tags.head()

The equivalent of a SQL `join` statement in Pandas is the `merge()` method.  We specify a left join, to keep movies without any tags, and indicate rows should be matched by index, which is the movie_id.

In [ ]:
df = df.merge(all_tags.to_frame(), left_index=True, right_index=True, how='left')
df.head()

We'll use the same sort of pipeline to one-hot encode the tags.  (It might be better to use an alternative encoding that accounts for the number of times each tag was applied.)  Then, a `FeatureUnion` can join the two one-hot encoded matrices.

In [ ]:
tag_pipe = Pipeline([('encoder', DictEncoder('tag')),
                     ('vectorizer', DictVectorizer())])
union = FeatureUnion([('categories', cat_pipe),
                      ('tags', tag_pipe)])

In [ ]:
features = union.fit_transform(df)
features

Over 16k features!

In [ ]:
nn = NearestNeighbors(n_neighbors=20).fit(features)

With the tags, we now clearly pick out *Toy Story 2* as the most similar to *Toy Story*.  But the rest seem worse.  We're not getting any of the Pixar movies, for instance.

In [ ]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

The problem becomes more clear looking at *Dr. Strangelove*.  *The Mouse that Roared* is a good pick, but most of the rest have no tags.  The problem is that the tag space is so large, and so sparsely populated, that even similar movies will have very few overlapping tags.  As a result, movies with no tags end up being the closest to most movies.

In [ ]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

## Dimensional Reduction

We don't actually expect there to be 16k different concepts expressed in the tags.  Rather, several tags may be associated with a single underlying concept.  In the above, we can see tags for *animation* and *cartoon*, for example.

There are several methods of **dimensional reduction** that attempt to pull out a lower-dimensional structure.  We'll be using Truncated Singular Value Decomposition (SVD), which works well with sparse matrices. SVD attempts to find orthogonal directions within feature space, along which the feature matrix has the most variation.  Recall that for $X$, an $n \times p$ feature matrix, we have the [Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) with

$$ X = U \Sigma V^T $$

where $U$ is a unitary $n \times n$ matrix, $\Sigma$ is a diagonal $n \times p$ matrix, and $V$ is a unitary $p \times p$ matrix.  With this decomposition, we can then truncate the diagonal matrix $\Sigma$ to include only the $m$ dimensions with the most variation.

Our choice of $m=100$ seems reasonable, but can be tuned to adjust the sensitivity to the tags.

In [ ]:
tag_pipe = Pipeline([('encoder', DictEncoder('tag')),
                     ('vectorizer', DictVectorizer()),
                     ('svd', TruncatedSVD(n_components=100))])
union = FeatureUnion([('categories', cat_pipe),
                      ('tags', tag_pipe)])

In [ ]:
features = union.fit_transform(df)
features

In [ ]:
nn = NearestNeighbors(n_neighbors=20).fit(features)
nn.fit(features)

Now we seem to be getting mostly Disney movies for *Toy Story*, which seems reasonable.

In [ ]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

Now, the algorithm finds movies like *The Great Dictator* and *MASH*, which are fairly similar in message to *Dr. Strangelove*.  It also finds *Full Metal Jacket*, another Stanley Kubrick war movie, albeit with a very different tone.

In [ ]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

We can examine the directions in feature space that the SVD picked out.  Here, we print the top ten tags associated with each dimension of the SVD.  We can see dimensions corresponding to classic movies, sci-fi franchises, comedies, and movies adapted from books.  (Note that the SVD has found that "based on a book", "adapted from:book", and "based on book" are all associated with each other.)

In [ ]:
svd = tag_pipe.named_steps['svd']
vect = tag_pipe.named_steps['vectorizer']
[", ".join([vect.feature_names_[i] for i in c.argsort()[:-10:-1]])
 for c in svd.components_]

## Recommendation for a User

The models so far seem to do a pretty good job of suggesting movies similar to a particular movie.  Sometimes, this is all that is needed.  If we notice a user buying a particular movie, we could use this to suggest similar movies they might like.

Often, however, we wish to suggest a movie to a user.  One approach would be to calculate an "average movie" that a particular user enjoyed, and then use the nearest neighbors approach to find movies similar to that one.

To help us judge, we'll add the movie titles to the ratings table.

In [ ]:
df_ratings_title = df_ratings.merge(df[['title']], left_on='movie_id', right_index=True)

And we'll look at user 9689, who seems to be a fan of horror.

In [ ]:
uid = 9689
df_ratings_title[df_ratings_title.user_id == uid].sort_values('rating', ascending=False)

We'll compute a weighted average movie, using the scores given as ratings.

In [ ]:
m_locs = df_ratings[df_ratings.user_id == uid]['movie_id'].apply(lambda x: df.index.get_loc(x))
scaled_ratings = df_ratings[df_ratings.user_id == uid]['rating'] * 0.2
weighted_avg_movie = scaled_ratings.values.reshape(1,-1).dot(features[m_locs,:].toarray()) / len(scaled_ratings)

And as expected, we get a bunch of horror movies.

In [ ]:
dists, indices = nn.kneighbors(weighted_avg_movie)
df.iloc[indices[0]]

## Cooperative Learning

Content-based recommendation systems work well to find items similar to those a user already likes.  To help with that, we turn to a collaborative learning model.  Instead of finding movies similar to movies a user liked, we will find *users* similar to a given user, and then find the movies that they rated highly.

How do we describe users?  We could use demographic information, but we have better set of features handy: the users' ratings.  Two users are similar if they have rated movies similarly.

In [ ]:
by_user_ratings = df_ratings.groupby('user_id').apply(
    lambda items: {i[1]: i[2] for i in items.itertuples()}) # 0 is index
features = DictVectorizer().fit_transform(by_user_ratings)

Previously, we have been using the default [Euclidean metric](https://en.wikipedia.org/wiki/Euclidean_distance) in the nearest neighbors calculation:

$$ d(x, y) =  \left| x - y \right|^2 \ . $$

For users, we will instead use the [cosine metric](https://en.wikipedia.org/wiki/Cosine_similarity),

$$ d(x, y) = 1 - \frac{ x\cdot y}{|x|\ |y|} \ , $$

which cares about angles between vectors in feature space.  This dependence on angle only lessens the effect of users using different scales to rate the movies

In [ ]:
nn = NearestNeighbors(n_neighbors=20, metric='cosine', algorithm='brute').fit(features)

In [ ]:
dists, indices = nn.kneighbors(features[by_user_ratings.index.get_loc(uid), :])
neighbors = [by_user_ratings.index[i] for i in indices[0]][1:]
ratings_grp = df_ratings_title[df_ratings_title['user_id'].isin(neighbors)] \
    .groupby('title')['rating']
len(ratings_grp)

There are a bunch of movies to consider, but we want to pick those that were most enjoyed by the neighbor users.  There's a few ways we might try.

### Attempt 1: Naive Mean

We could pick those that have the highest average rating, but these are typically movies reviewed by only one or two users.

In [ ]:
ratings_grp.agg(['mean', 'count']).sort_values('mean', ascending=False)

### Attempt 2: Naive Sum

Using the sum of the ratings will solve this particular problem, but will tend to highlight only those popular movies that almost everyone has seen.  There's no chance for a movie that's a hit with only some of the neighbors to shine through.

In [ ]:
ratings_grp.agg(['sum', 'count']).sort_values('sum', ascending=False)

### Attempt 3: Bayesian Smoothing

One way to balance these two competing needs is to introduce a **Bayesian prior**.  This is the estimate we make that every movie has a rating $\mu$, before seeing any of the user ratings.  As we see ratings from users come it, we formulate a **posterior** estimate of the rating, taking into account both the prior and the new information provided by the ratings.  The posterior estimate of the rating, after seeing the $n$ reviews $\{x_i\}$, can be written as

$$ \frac{ \sum_{i=0}^n x_i + \mu N}{n + N} \ , $$

where $N$ reflects our level of confidence in the prior.  Adjusting $N$ affects the number of user reviews needed to push the posterior estimate away from $\mu$.  Adjusting $\mu$ affects where in the rankings movies with few reviews appear.  This technique is known as [Bayesian Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing).

In this case, we use $\mu = 3$, $N = 5$, essentially starting each movie off with five reviews of three stars.  The top rated movies are all ones reviewed by our user.  Looking a little bit down the list, we see a recommendation for *The Thing*, which seems very reasonable.  *Ghostbusters* also ranks highly.  This is somewhat surprising, since our user didn't rate any comedies.  Evidently, other fans of horror films like *Ghostbusters* anyway, so maybe our user should give it a try.

In [ ]:
def bayes_sum(N, mu):
    return lambda x: (x.sum() + mu*N) / (x.count() + N)

ratings_grp.aggregate(bayes_sum(5, 3)).sort_values(ascending=False)

**Exercises:**

1. Explore the effect of different distance metrics on the recommendations.  The 'euclidean', 'manhattan', and 'cosine' metrics are some of the more common ones.  Check the `NearestNeighbors` documentation for additional options.

2. Add in the year as a feature of movies.  Should it be a continuous or categorical variable?  While it's likely that a user who likes movies from 1970 will also like movies from 1971 (suggesting a continuous feature), a user liking movies from 1970 and 1990 is no guarantee that they like movies from 1980 (suggesting a categorical feature).  Can you find an encoding that's in between these two options?  How does the scale of the year, if treated as a continuous feature, affect the KNN calculation?  You may want to use a `StandardScaler`, from `sklearn.preprocessing`, to reduce this effect.

3. In weighting reviews, we consider not reviewing a movie to be less of a recommendation that scoring it a one.  Shift the rating scale to change this, and see how the affects the resultant recommendations.

## Regression of Ratings

We have developed several ways to recommend movies to a user.  However, it is hard to tell how well these systems work, since most users have not rated most movies.  We may return a list of recommendations, none of which has been reviewed by the user.  Is that a sign the user doesn't like those movies, or that they just haven't seen them yet?

An alternative approach is develop a regression model to learn the ratings.  That is, given a user $u$ and an item $i$, we wish to build a model to predict $r_{ui}$, the rating given by $u$ to $i$.  In evaluating the performance of the model, we will consider only user/items pairs where a rating exists.  Our predictions, $\hat r_{ui}$ can be used to determine which unrated items may be preferred by a user.

In order to test how well we can predict rating that haven't been made, we train on only a portion of the data and then evaulate its perfomance on the remainder of the data.  We will take care to use a train-test split.  We will use the root mean-squared error (RMSE) to evaulate our model.  That is, we will compute

$$ \mbox{RMSE} = \sqrt{\frac{1}{n} \sum_{ui} \left(\hat r_{ui} - r_{ui}\right)^2} $$

where $r_{ui}$ are the ratings for $n$ $ui$-pairs in the test set.
  

For the purposes of having code execute quickly for demonstration, we will work with only a small portion of the users.

In [ ]:
users = df_ratings['user_id'].unique()
np.random.seed(42)
user_sample = np.random.choice(users, 3000)
df_ratings_sample = df_ratings[df_ratings['user_id'].isin(user_sample)]
X_train, X_test, y_train, y_test = train_test_split(df_ratings_sample, df_ratings_sample['rating'])

## Baseline Model

We start with a baseline model that accounts for the fact that different users and different items will have different baseline scores.  This model has no interaction between the user and item:

$$ b_{ui} = \mu + b_u + b_i \ . $$

This may seem silly to worry about, but the baseline model is very important in practice.  The [winners of the Netflix Grand Prize](http://netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) succeeded largely on the strength of their baseline model.  Alone, it performed nearly as well as the existing Netflix model.

That model included a number of time-dependent terms, but we will consider each of the terms to be constant.  As such $\mu$ will be the average overall rating, $b_u$ will be the average difference from the global mean of all ratings by user $u$, and $b_i$ will be the average difference from the global mean of all ratings of item $i$.  If we one-hot encode both the users and movies, we can find these coefficients with a linear regression.

In [ ]:
class Dictizer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.col].apply(lambda x: {x: 1})

In [ ]:
user_dict = Pipeline([('dict', Dictizer('user_id')),
                      ('vect', DictVectorizer())])
movie_dict = Pipeline([('dict', Dictizer('movie_id')),
                       ('vect', DictVectorizer())])
union = FeatureUnion([('user_dict', user_dict),
                      ('movie_dict', movie_dict)])
lr_pipe = Pipeline([('union', union),
                    ('lr', Ridge(alpha=0))])

In [ ]:
lr_pipe.fit(X_train, y_train)

A small function will help us calculate RMSE scores.

In [ ]:
def rmse(model, X, y):
    return np.sqrt(mean_squared_error(model.predict(X), y))

To callibrate our expectations, the RMSE of the mean model is the square root of the variance.

In [ ]:
np.sqrt(y_test.var())

Our baseline model does about 18% better.

In [ ]:
rmse(lr_pipe, X_test, y_test)

However, our model scores significantly better on the training set.

In [ ]:
rmse(lr_pipe, X_train, y_train)

In [ ]:
len(lr_pipe.named_steps['lr'].coef_)

## Overfitting and Cross-validation

This discrepancy is a sign that we may be overfitting.  Training data is never perfectly clean.  It contains both the underlying signal, which you want to model, and some random noise, which you do not.  Since our model has over 11K parameters, it has plenty of flexibility to fit the noise as well as the signal.

The noise in this case is the same issue that prompted us to use a Bayesian mean to rate the movies suggested by neighbor users.  Prior to seeing any ratings for a item, for example, we would expect $b_i = 0$.  After seeing a single rating $r$, we shouldn't set $b_i = r - mu$.  Instead, we should update our estimate or $b_i$ somewhat in between these values, depending on our confidence in the prior.

While we could add in these prior ratings by hand, we can use regularization to produce the same effect.  Recall that in ridge regression, we add a term proportional to $\beta^2$ to the quantity being minimized:

$$ \left|y - X\cdot\beta\right|^2 + \alpha\ \left|\beta\right|^2 \ . $$

If the model is being used to take an average, as happens with one-hot encoding, the resultant coefficients will be

$$ \hat\beta_i = \frac{\sum_{j\mid X_{ij}=1} y_j}{n + \alpha} \ , $$

where $n$ is the number of $y_j$ included in the sum.  Note that this is equivalent to our previous Bayesian mean, with $\mu=0$ and $N=\alpha$.  We find the optimal value of $\alpha$ through cross-validation and a grid search.

In [ ]:
gs = GridSearchCV(lr_pipe, {'lr__alpha': np.logspace(-1,1,5)}, n_jobs=-1)

`GridSearchCV` is itself an estimator.  When it is fit, it will run 3-fold cross validation (by default) on each set of hyperparameters, choose the best set of hyperparameters, and then train the estimator with the best hyperparameters on the full set of data.

In [ ]:
gs.fit(X_train, y_train)

We see that performance on the test set has improved somewhat.  (The predict method on the grid search object simply calls predict on the best fit estimator.)

In [ ]:
rmse(gs, X_test, y_test)

The best estimator, and its hyperparameter, are available as properties of the grid search object.  (Scikit Learn uses the convention that a trailing underscore indicates a property available only after a model has been fit.)

In [ ]:
best_lr = gs.best_estimator_
gs.best_params_

## Modeling Interaction

With a baseline model for each user and item, we now look to build a model that accounts for the residual left over.  That is, we wish to model

$$ d_{ui} \equiv r_{ui} - b_{ui} \ . $$

The current state of the art for iteraction modeling is **matrix factorization**.  Taking a cue from our previous work with dimensional reduction, we suppose that there are a small number of factors $f$ that users like about movies.  Thus, the interaction can be understood by figuring out the mapping of movies to those factors and those factors to user preferences.  In other words, we assume we can write

$$ d_{ui} = \sum_f U_{uf} M_{fi} \ , $$

where $U$ records what factors each user prefers and $M$ records which factors are in each movie.  As with linear regression, the goal is to minimize the sum of squared errors

$$ \sum_{u,i} \left(d_{ui} - \sum_f U_{uf} M_{fi} \right)^2 \ . $$

The values in both $U$ and $M$ are unknown.  We can work to estimate them through a procedure called **alternating least squares**.  If we assume that $M$ is known, we can find $U$ through a least squares optimization much like that of linear regression:

$$ U = d\ M^T (M\ M^T)^{-1} \ . $$

Then, treating $U$ as known, we can solve for $M$:

$$ M = (U^T\ U)^{-1} U^T\ d \ . $$

Iterating this process should take us closer to the optimal solution.

To avoid the potential of overfitting, we add a regularization parameter $\alpha$ to the algorithm.  The quantity to be minimized becomes

$$ \sum_{u,i} \left(d_{ui} - \sum_f U_{uf} M_{fi} \right)^2 + \alpha \left( \sum_{u,f} U_{uf}^2 + \sum_{f,i} M_{fi}^2 \right) \ . $$

However, most of the ratings, and therefore the residuals $d_{ui}$, are unknown.  Thus, we restrict the sum to include only those pairs of $u,i$ for which $r_{ui}$ is defined.

This algorithm is implemented in the `ResidualMatixFactorization` class.  It takes a baseline estimator, from which the residuals are calculated, and three hyperparameters:
- `n_factors` is the number of underlying factors.
- `n_iters` is the number of iterations of the alternating least squares algorithm to run.
- `alpha` is the regularization parameter.

In [ ]:
# Algorithm adapted from http://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/

class ResidualMatrixFactorization(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self, base_est, n_factors, n_iters, alpha):
        self.base_est = base_est
        self.n_factors = n_factors
        self.n_iters = n_iters
        self.alpha = alpha
        
        self.user_index = self.rating_index = self.Q = self.W = self.user_f = self.movie_f = None
    
    def fit_step(self):
        if self.user_index is None:
            self.init_fit(X, y)
        
        alphas = self.alpha * np.eye(self.n_factors)
        for u, Wu in enumerate(self.W):
            movie_f = self.movie_f[:,Wu]
            self.user_f[u] = np.linalg.solve(
                np.dot(movie_f, movie_f.T) + alphas,
                np.dot(movie_f, self.Q[u, Wu])).T
        for i, Wi in enumerate(self.W.T):
            user_f = self.user_f[Wi,:]
            self.movie_f[:, i] = np.linalg.solve(
                np.dot(user_f.T, user_f) + alphas,
                np.dot(user_f.T, self.Q[Wi, i]))
        
        return self
    
    def fit(self, X, y):
        self.base_est.fit(X, y)
        residuals = y - self.base_est.predict(X)
        df = pd.DataFrame({'user_id': X['user_id'], 'movie_id': X['movie_id'], 'residuals': residuals})
        
        rating_mat = df.pivot_table('residuals', 'user_id', 'movie_id')
        self.user_index = rating_mat.index
        self.movie_index = rating_mat.columns
        
        self.Q = rating_mat.fillna(0).values
        self.W = (~rating_mat.isnull()).values
        self.user_f = np.random.rand(self.Q.shape[0], self.n_factors)
        self.movie_f = np.random.rand(self.n_factors, self.Q.shape[1])

        for n in xrange(self.n_iters):
            self.fit_step()
        
        return self
    
    def interaction_prediction(self, row):
        uid = row['user_id']
        mid = row['movie_id']
        if uid in self.user_index and mid in self.movie_index:
            return np.dot(self.user_f[self.user_index.get_loc(uid), :],
                          self.movie_f[:, self.movie_index.get_loc(mid)])
        return 0
        
    def predict(self, X):
        return self.base_est.predict(X) + X.apply(self.interaction_prediction, axis=1).values

We need to set the regularization parameter through cross validation.

In [ ]:
gs = GridSearchCV(ResidualMatrixFactorization(best_lr, 20, 10, 1),
                  {'alpha': [1,3,10,30,100]})
gs.fit(X_train, y_train)
gs.best_params_

It looks like we gain about several percent on our baseline model.  This may not seem like much, but the Netflix Grand Prize winners managed less than a 10% improvement on their baseline model.

In [ ]:
rmse(gs, X_test, y_test)

The number of factors can also be explored through cross-validation.  As this parameter slows the model significantly, we must weigh the execution time of the model along with its RMSE score.  Properly speaking, we should be exploring both hyperparameters at the same time.  However, this would increase the number of combinations to be tested.  Optimizing hyperparameters independently often yields values near the global optimum.

In [ ]:
gs = GridSearchCV(ResidualMatrixFactorization(best_lr, 20, 10, 30),
                  {'n_factors': [10, 20, 50, 100]})
gs.fit(X_train, y_train)
gs.best_params_

A larger number of factors is better, but the improvement is marginal.  We'll stick with $n=20$ for performance.

In [ ]:
rmse(gs, X_test, y_test)

We have been using 10 iterations, somewhat arbitrarily.  Let's see how the errors are changing each step.

In [ ]:
rest = ResidualMatrixFactorization(best_lr, 20, 0, 30)
rest.fit(X_train, y_train)
errors = []
for i in xrange(10):
    rest.fit_step()
    errors.append((rmse(rest, X_test, y_test), rmse(rest, X_train, y_train)))
plt.plot(errors)
plt.legend(['Out-of-sample error', 'In-sample error'])

We're probably getting most of the available performance in those first 10 iterations.

Finally, let's see what we would recommend to user 9689 based on these ratings.

In [ ]:
predict_df = pd.DataFrame({'movie_id': X_train['movie_id'].unique(), 'user_id': 9689})
predict_df = predict_df.merge(df[['title']], left_on='movie_id', right_index=True)
predict_df['scores'] = rest.predict(predict_df)
predict_df.sort_values('scores', ascending=False).head(20)

**Exercises:**

1. Implement another user-item interaction model.  Here are a few ideas:
   - Measure how similar the user is to fans of the item, and regress against that distance.
   - Measure how similar the item is to items the user likes, and regress against that distance.
   - Use a nearest neighbors approach to report the scores given to an item by users similar to the user in question.
   
2. Blend two (or more) interaction models together.  These can be different algorithms from (1) or several matrix factorization algorithms with different hyperparameters.  Linear regression is a good way to automatically perform a weighted average together.  The BellKor team had success with **gradient-boosted trees** as blenders.

3. Add features of the movie or user to the blend.  A nonlinear model, such as **random forest** or gradient-boosted trees, may be able to detect that one of the component models does particularly well for a certain subset of features.

## References

- ["Recommendation Systems", from *Mining of Massive Datasets*](http://infolab.stanford.edu/~ullman/mmds/ch9.pdf), by Jure Leskovec, Anand Rajaraman, and Jeff Ullman
- ["Recommender Systems"](http://vikas.sindhwani.org/recommender.pdf), by Prem Melville and Vinkas Sindhwani
- [How to Build a Recommendation System](http://www.mickaellegal.com/blog/2014/1/30/how-to-build-a-recommender), by Mickaël Le Gal
- ["How to Build a Recommender System"](http://blogs.gartner.com/martin-kihn/how-to-build-a-recommender-system-in-python/), by Martin Kihn
- ["The BellKor Solution to the Netflix Grand Prize"](http://netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf), by Yehuda Koren
- ["Netflix Update: Try This at Home"](http://sifter.org/~simon/journal/20061211.html), by Simon Funk
- ["Alternating Least Squares Method for Collaborative Filtering"](http://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/), by Bugra Akyildiz

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*